In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re

In [2]:
driver = webdriver.Chrome(executable_path=r"C:\Users\Mark\Anaconda3\selenium\webdriver\chrome\chromedriver.exe")

In [2]:
df_beer = pd.read_csv('./df_beer.csv', encoding = "latin1")

In [7]:
# scraper stopped running at this point
df_beer['url'][46:]

46         /beer/profile/345/1005/
47       /beer/profile/1199/58914/
48         /beer/profile/63/34420/
49       /beer/profile/1146/57747/
50         /beer/profile/306/1212/
51        /beer/profile/199/48505/
52         /beer/profile/396/3833/
53         /beer/profile/418/3434/
54         /beer/profile/287/1094/
55           /beer/profile/45/680/
56       /beer/profile/2210/38180/
57          /beer/profile/113/571/
58       /beer/profile/43153/1056/
59        /beer/profile/345/99873/
60       /beer/profile/2210/41815/
61          /beer/profile/64/6260/
62         /beer/profile/64/18862/
63           /beer/profile/39/131/
64            /beer/profile/26/39/
65         /beer/profile/182/1351/
66           /beer/profile/73/226/
67          /beer/profile/42/3457/
68         /beer/profile/313/1545/
69        /beer/profile/863/22227/
70       /beer/profile/1199/34146/
71            /beer/profile/29/65/
72        /beer/profile/140/55939/
73         /beer/profile/287/1444/
74           /beer/p

In [4]:
review_grabber = lambda x: re.search('overall:\s\d\.\d*(.*[\s\S]*\d characters)', x).group(1)

def review_cleaner(string):
    x = review_grabber(string)
    x = re.sub('\d* characters', '', x)
    x = re.sub('\n', ' ', x)
    x = re.sub(',', '', x)
    return x
    
def get_user_reviews_csv(url):
    counter = 0
    review_number = 1
    for i in range(0,101):
        driver.get(f'https://www.beeradvocate.com{url}?view=beer&sort=&start={i*25}')
        sleep(2)
        beer_page = driver.page_source
        beer_page_soup = BeautifulSoup(beer_page, 'lxml')
        
        reviews = beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})
        
        counter += 1
        print(f'{url} -- page {counter}')
        
        for count, review in enumerate(reviews):
            score = review.find('span', {'class': 'BAscore_norm'}).text
            breakdown = review.find('span', {'class': 'muted'}).text
            u_names = review.find('a', {'class':'username'}).text
            try:
                r_text = review_cleaner(reviews[count].text)
            except:
                r_text = "No Review"
                
            master_list = [str(review_number), url, score, breakdown, u_names, r_text]
            with open('./aws_user_reviews.csv', 'a+') as f:
                print(','.join(master_list), file=f)
            
            review_number += 1
    
    sleep(2)

In [5]:
df_beer['url'][0:2].apply(get_user_reviews_csv)

/beer/profile/1199/11757/ -- page 1
/beer/profile/1199/11757/ -- page 2
/beer/profile/64/2093/ -- page 1
/beer/profile/64/2093/ -- page 2


0    None
1    None
Name: url, dtype: object